In [2]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [3]:
import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38') 

2023-03-31 16:47:03.111 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0296:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230331-1646-0.2.109-b71b065e4bb6.log


In [4]:
from hail.ggplot import *

import plotly
import plotly.io as pio
pio.renderers.default='iframe'

In [29]:
full = hl.vds.read_vds('/net/ascratch/people/plggosborcz/full-rpkm.vds')
full = hl.vds.to_dense_mt(full)

full = full.naive_coalesce(2000)
full.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/full-dense.mt')

mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/full-dense.mt')
mt = mt.annotate_entries(
    GT = hl.vds.lgt_to_gt(mt.LGT, mt.LA)
)
mt.write('/net/ascratch/people/plggosborcz/mt-gt.mt')

In [171]:
mt = hl.read_matrix_table('/net/ascratch/people/plggosborcz/mt-gt.mt')

In [5]:
mt = mt.filter_cols(mt.s != '494') #one sample removed due to quality issues
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

# filter by gnomad coverage
cov = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/gnomad/gnomad.genomes.r3.0.1.coverage.ht/')
cov = cov.filter(cov.over_1 > 0.9)
mt = mt.filter_rows(hl.is_defined(cov[mt.locus]), keep = True)

mt = mt.annotate_cols(
    group = hl.if_else(
        mt.s.contains('B'),
        'local_controls',
        hl.if_else(
            (mt.s.contains('NA') | mt.s.contains('HG')),
            '1kg_controls',
            'GTS'
        )
    )
)   

#perform per_group variant QC - dp and gq stats + hwe (for GTS and sportsmen only for now)
mt = mt.annotate_rows(dp_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.DP)),
                      gq_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.GQ)),
                      hwe = hl.agg.group_by(mt.group, hl.agg.hardy_weinberg_test(mt.GT)))

mt = mt.annotate_rows(n_below_dp_3 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.DP < 3)),
                      n_below_gq_30 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GQ <30)))

mt = mt.filter_rows((mt.dp_qc.get('GTS', hl.struct(mean=0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.dp_qc.get('local_controls', hl.struct(mean=0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 5) &
                    (mt.gq_qc.get('GTS', hl.struct(mean=0.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 40) &
                    (mt.gq_qc.get('local_controls', hl.struct(mean=0.0, stdev=0.0, min=0.0, max=0.0, n=0, sum=0.0)).mean > 40) &
                    (mt.hwe.get('GTS', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 1E-15) &
                    (mt.hwe.get('local_controls', hl.struct(het_freq_hwe=0.0, p_value=0.5)).p_value > 1E-15) &
                    (mt.n_below_dp_3.get('local_controls', 0) < 8) &
                    (mt.n_below_gq_30.get('local_controls', 0) < 40) &
                    (mt.n_below_dp_3.get('GTS', 0) < 8) &
                    (mt.n_below_gq_30.get('GTS', 0) < 340))

#perform sample and overall variant_qc
mt = hl.variant_qc(mt)
mt = hl.sample_qc(mt)

#this is still to be run
mt = mt.filter_cols(
    (mt.sample_qc.n_singleton < 100000),
    (mt.sample_qc.r_het_hom_var > 3.3),
    (mt.samples_qc.call_rate > 0.9) #this is to implement still
)

mt = hl.split_multi_hts(mt)

In [6]:
mt.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-split.mt')

2023-03-31 11:42:56.089 Hail: INFO: wrote matrix table with 10733631 rows and 788 columns in 1995 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-split.mt


In [9]:
gnomad = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/gnomad/gnomad.genomes.v3.1.1.sites.ht/')
cadd = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/cadd.ht')
vep = hl.read_table('/net/pr2/projects/plgrid/plggneuromol/resources/vep38/grch38_context_vep_annotated.ht')

In [11]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-split.mt')

mt = mt.annotate_rows(gnomad_v_3_1 = gnomad[mt.row_key])
mt = mt.annotate_rows(cadd = cadd[mt.row_key])
mt = mt.annotate_rows(vep = vep[mt.row_key])

In [13]:
mt.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-anno.mt')

2023-03-31 17:23:06.462 Hail: INFO: wrote matrix table with 10733631 rows and 788 columns in 1995 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-anno.mt
